In [1]:
#!/usr/bin/env python

"""
12/26/2017  carolyn.parkinson@gmail.com

After preprocessing, this script does the following for each subject:
    1) Merges the preprocessed time series data across the 6 runs
    2) Extracts this average preprocessed time series from each anatomical ROI
    3) Saves the average preprocessed time series for each ROI in a dictionary
    where the keys are the ROI labels and the values are the corresponding time
    series.
"""

# import relevant python modules
import os
import shutil
import subprocess
import json
import glob
import csv
import numpy as np

In [6]:
# set path to directory containing data for all subjects
mypath = ('/Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts')

# read in the list of subjects in the study whose data we'll be preprocesssing
#list_file = '{}/subjects.json'.format(mypath)
#with open(list_file) as data_file:
#    subj_list = json.load(data_file)

subj_list = ["sub-289"]
# define a function to execute shell commands
def sh(c):
    '''
    run shell commands
    '''
    subprocess.call(c, shell = True)
    #print(c)

# define a function to write dictionaries to files with json encoding
def ld_writeDicts(filePath,dict):
    f = open(filePath,'w')
    newData = json.dumps(dict, sort_keys = True, indent = 4)
    f.write(newData)
    f.close()


In [8]:
for subject in subj_list:

    print(mypath)
    print(subject)
    # specify and enter directory containing this subject's nifti files
    nifti_dir = "{}/nifti/{}".format(mypath, subject)
    os.chdir(nifti_dir)

    # also specify directory containing their SUMA files
    suma_dir = "{}/{}/SUMA".format(mypath, subject)

    # copy each run of preprocessed data into the subject's main nifti directory
    epi_fnames = {}
    for epi_num in range(1,5):
        # specify the prefix for the preprocessed file names for this run; this is
        # also the name of the AFNI-generated sub-directory containing these files
        epi_prefix = 'epi{}_preprocessed'.format(epi_num)
        # specify file name of preprocessed data within that sub-directory
        epi_fname = '{}.cleanEPI+orig'.format(epi_prefix)
        # create a dictionary where  keys are the run names ('epi1', 'epi2', etc.)
        # and values are the corresponding full file names (to iterate through later)
        epi_fnames['epi{}'.format(epi_num)] = epi_fname
        # specify and execute an AFNI command to copy these files
        cp_cmd = '3dcopy {}/{}/{} {}/{}.cleanEPI'.format(nifti_dir, epi_prefix, epi_fname, nifti_dir, epi_prefix)
        sh(cp_cmd)

    # specify what the file of concatenated data should be called
    all_epi = "{}_epi_all_preprocessed".format(subject)
    # specify and execute an AFNI command to concatenate all runs' preprocessed data
    tcat_cmd = ("3dTcat -session {} -prefix {} "
                "{epi1} {epi2} {epi3} {epi4} {epi5} {epi6}").format(nifti_dir, all_epi,  **epi_fnames)
    sh(tcat_cmd)

    # Clean up directory a bit
    # delete redundant copies of preprocessed data for each run
    for num, fname in epi_fnames.iteritems():
        rm_cmd = 'rm {}/{}*'.format(nifti_dir, fname)
        sh(rm_cmd)

    # also move raw epi data into its own directory
    raw_epi_dir = '{}/raw_epi_data'.format(nifti_dir)
    if not os.path.exists(raw_epi_dir):
        os.makedirs(raw_epi_dir) # create this directory if it doesn't exist already
    mv_cmd = 'mv {}/{}_fMRI_* {}'.format(nifti_dir, subject, raw_epi_dir)
    sh(mv_cmd)

    # specify table of ROI labels from Freesurfer cortical parcellation
    lookup_table = "{}/aparc+aseg_rank.niml.lt".format(suma_dir)

    # create a dictionary to fill that matches ROI numbers to labels
    roi_dict = {}

    with open(lookup_table) as csvfile:
        # skip first 4 lines (header)
        next(csvfile)
        next(csvfile)
        next(csvfile)
        next(csvfile)
        reader = csv.DictReader(csvfile, delimiter = " ", fieldnames = ["roi_num", "roi_label"])
        for row in reader:
            roi_dict[row['roi_num']] = row['roi_label']

    # save as text file with json encoding
    ld_writeDicts('{}_roi_dict_dk_atlas.json'.format(subject), roi_dict)

    # Now get masks of cortical areas from Freesurfer parcellation so that we can
    # extract the average time series from each mask.

    # Get the total number of ROIs in this subject's parcellation file; this is
    # 1 less than the length of the lookup table due to the file footer. The
    # length of different subjects' lookup tables may vary a bit, as some
    # subjects have a 5th ventricle in the FreeSurfer subcortical segmentation;
    # see http://surfer.nmr.mgh.harvard.edu/fswiki/SubcorticalSegmentation/
    all_roi_num = range(1, (len(roi_dict)-1))

    # get Freesurfer parcellation file that's been aligned to our anatomical scan
    aparc = "{}/aparc+aseg_rank_Alnd_Exp".format(nifti_dir)

    # resample parcellation file to functional resolution
    resamp_cmd = ("3dresample -master {}+orig -rmode NN "
                  "-input {}+orig -prefix {}_3mm").format(all_epi, aparc, aparc)
    sh(resamp_cmd)

    # dictionary to store preprocessed time series indexed by ROI
    subj_ts_dict = {}

    for roi_id in all_roi_num:
        # extract ROI label form ROI dictionary
        label_roi = roi_dict[str(roi_id)]

        # Extract the average time series across voxels that ROI
        avg_cmd = ("""3dmaskave -quiet -mask {}_3mm+orig"<{}..{}>" {}+orig > avg_{}.1D""").format(aparc, roi_id, roi_id, all_epi, label_roi)
        sh(avg_cmd)

        # Then read in the 1D file generated by the above command (average time
        # series across voxels within a given ROI) and add it to a dictionary
        # indexed by ROI label
        this_vec = "avg_{}.1D".format(label_roi)
        subj_ts_dict[label_roi] = np.loadtxt(this_vec).tolist()

    # save dictionary to a text file with json encoding
    ld_writeDicts('{}_roi_dk_atlas_ts_dict.json'.format(subject), subj_ts_dict)

    # move the individual ROI-wise time series 1D files to their own sub-directory
    sub_dir = "{}/avg".format(nifti_dir)
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir) # create qvecs if it doesn't exist
    mv_sub_cmd = "mv avg*1D avg/"
    sh(mv_sub_cmd)


/Users/mdclark/Desktop/ISCAnalysis/data/fmri/ts
sub-289


KeyError: 'epi5'